In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
import torch.nn.functional as F

#from datasets import get_mnist_dataset, get_data_loader
#from utils import *
#from models import *

import pickle as pkl
import os

from generate_dataloaders import *

## Get Dataloaders

In [3]:
def get_dataloaders(train_filename,val_filename):
    path = os.getcwd()
    data_dir = path + '/data/'
    train_dataloader = pkl.load(open(data_dir + train_filename,'rb'))
    val_dataloader = pkl.load(open(data_dir + val_filename,'rb'))
    return train_dataloader,val_dataloader

In [2]:
path = os.getcwd()
data_dir = path + '/data/'

In [4]:
train_loader,val_loader = get_dataloaders('train_dataloader.p','val_dataloader.p')

In [5]:
ground_truth_dataloader = pkl.load(open(data_dir + 'ground_truth_dataloader.p','rb'))

In [6]:
print(torch.__version__)

1.3.1


## Scratchwork (IGNORE)

In [7]:
for i,x in enumerate(train_loader):
    print(len(x[0]))
    break

32


In [8]:
minibatch = torch.tensor([
                            [[1,2,3,4,5],[3,3,3,3,3],[1,1,1,1,1],[2,1,2,1,2]],
                            [[0,1,0,1,0],[1,1,1,1,1],[2,0,0,0,0],[0,0,0,0,2]]
                         ], dtype=torch.float32)

flagged_indices = torch.tensor([1,2])

upweight_value = 10

print(minibatch.shape)
print(minibatch)

print(flagged_indices.shape)
print(flagged_indices)

torch.Size([2, 4, 5])
tensor([[[1., 2., 3., 4., 5.],
         [3., 3., 3., 3., 3.],
         [1., 1., 1., 1., 1.],
         [2., 1., 2., 1., 2.]],

        [[0., 1., 0., 1., 0.],
         [1., 1., 1., 1., 1.],
         [2., 0., 0., 0., 0.],
         [0., 0., 0., 0., 2.]]])
torch.Size([2])
tensor([1, 2])


In [9]:
batch_size, num_tokens, emb_dim = minibatch.shape
print(type(minibatch))
minibatch[range(batch_size),flagged_indices,:] *= upweight_value
print(batch_size, num_tokens, emb_dim)
minibatch

<class 'torch.Tensor'>
2 4 5


tensor([[[ 1.,  2.,  3.,  4.,  5.],
         [30., 30., 30., 30., 30.],
         [ 1.,  1.,  1.,  1.,  1.],
         [ 2.,  1.,  2.,  1.,  2.]],

        [[ 0.,  1.,  0.,  1.,  0.],
         [ 1.,  1.,  1.,  1.,  1.],
         [20.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  2.]]])

In [10]:
minibatch.sum(1) / (num_tokens + upweight_value - 1)

tensor([[2.6154, 2.6154, 2.7692, 2.7692, 2.9231],
        [1.6154, 0.1538, 0.0769, 0.1538, 0.2308]])

In [11]:
print(type(minibatch))

<class 'torch.Tensor'>


In [12]:
embed = torch.tensor(np.array([[2,4,5,6],[1,3,45,7],[3,4,5,6]]))

In [13]:
centers = torch.tensor(np.array(([2,3,4,5],[1,2,4,5])))

In [14]:
torch.sum((embed[:,None,:]-centers)**2,2)

tensor([[   3,    7],
        [1686, 1686],
        [   4,   10]])

In [15]:
cluster_distances, cluster_assignments = torch.sum((embed[:,None,:]-centers)**2, 2).min(1)
cluster_assignments

tensor([0, 1, 0])

In [16]:
for i, (tokens, labels, flagged_indices) in enumerate(train_loader):
    print(tokens, labels, flagged_indices)
    break

tensor([[   34,    19,  3042,   165,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [ 1901,   257,   171,   664,    85,    41,  2684,    20,   519,   182,
            41,   643,   214,   170,    30,    41,   958,  1789,   214,  8043,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [ 4418,    69,    60,   518,    11,    20,   216,    10,    84,     7,
            18,  4419,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [   68,   302,    60,  1207,  2449,   135,   398,    52,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [  360,    20,    78,    20,   532,     

## Neural Network Class

NOTE: Data loader is defined as:
- tuple: (tokens, flagged_index, problematic)

In [17]:
class neuralNetBow(nn.Module):
    """
    BagOfWords classification model
    """
    # NOTE: we can't use linear layer until we take weighted average, otherwise it will
    # remember certain positions incorrectly (ie, 4th word has bigger weights vs 7th word)
    def __init__(self, vocab_size, emb_dim, upweight=10):
        super(neuralNetBow, self).__init__()
        self.embed = nn.Embedding(vocab_size, emb_dim, padding_idx=2)
        self.upweight = upweight
    
    def forward(self, tokens, flagged_index):
        batch_size, num_tokens = tokens.shape
        embedding = self.embed(tokens)
#         print(embedding.shape) # below assumes "batch_size x num_tokens x Emb_dim" (VERIFY)
        
        # upweight by flagged_index
#         print(type(embedding))
        embedding[torch.LongTensor(range(batch_size)),flagged_index.type(torch.LongTensor),:] *= self.upweight
        
        # average across embeddings
        embedding_ave = embedding.sum(1) / (num_tokens + self.upweight - 1)
        
        return embedding_ave

### Clustering Stuff (un-tailored)

In [18]:
class KMeansCriterion(nn.Module):
    
    def __init__(self, lmbda):
        super().__init__()
        self.lmbda = lmbda
    
    def forward(self, embeddings, centroids):
        distances = torch.sum((embeddings[:, None, :] - centroids)**2, 2)
        cluster_distances, cluster_assignments = distances.min(1)
        loss = self.lmbda * cluster_distances.sum()
        return loss, cluster_assignments

In [66]:
def centroid_init(k, d, dataloader, model):
    ## Here we ideally don't want to do randomized/zero initialization
    centroid_sums = torch.zeros(k, d)
    centroid_counts = torch.zeros(k)
    for (tokens, labels, flagged_indices) in dataloader:
#         cluster_assignments = torch.LongTensor(tokens.size(0)).random_(k)
        cluster_assignments = labels
        
        model.eval()
        sentence_embed = model(tokens,flagged_indices)
    
        update_clusters(centroid_sums, centroid_counts,
                        cluster_assignments, sentence_embed)
    
    centroid_means = centroid_sums / centroid_counts[:, None]
    return centroid_means.clone()

def update_clusters(centroid_sums, centroid_counts,
                    cluster_assignments, embeddings):
    k = centroid_sums.size(0)

    centroid_sums.index_add_(0, cluster_assignments, embeddings)
    
    np_counts = np.bincount(cluster_assignments.data.numpy(), minlength=k)
    centroid_counts.add_(torch.FloatTensor(np_counts))

### Training Function (un-tailored, needs alterations)

In [166]:
def train_model(model, centroids, criterion, optimizer, train_loader, valid_loader, num_epochs=10, path_to_save=None, print_every = 100):

    model.train()
    num_gpus = torch.cuda.device_count()
    if num_gpus > 0:
        current_device = 'cuda'
    else:
        current_device = 'cpu'
    
    
    for epoch in range(num_epochs):
        k, d = centroids.size()
        centroid_sums = torch.zeros_like(centroids)
        centroid_counts = torch.zeros(k)
        total_epoch_loss = 0

        # run one epoch of gradient descent on autoencoders wrt centroids
        for i, (tokens, labels, flagged_indices) in tqdm(enumerate(train_loader)):
            tokens = tokens.to(current_device)
            labels = labels.to(current_device)
            flagged_indices = flagged_indices.to(current_device)

            # forward pass and compute loss
            sentence_embed = model(tokens,flagged_indices)
            cluster_loss, cluster_assignments = criterion(sentence_embed, centroids)

            # run update step
            optimizer.zero_grad()
            cluster_loss.backward(retain_graph=True)
            optimizer.step()
            
            #Add loss to the epoch loss
            total_epoch_loss += cluster_loss.data

            # store centroid sums and counts in memory for later centering
            update_clusters(centroid_sums, centroid_counts,
                            cluster_assignments, sentence_embed)

            if i % print_every == 0:
                losses = cluster_loss.data/len(tokens)
                print('Average training Loss at batch',i,': %.3f' % losses)
                
            
        total_epoch_loss /= len(train_loader.dataset)
        
        print('Average training Loss after epoch',epoch,': %.3f' % total_epoch_loss)

        # update centroids based on assignments from autoencoders
        centroids = centroid_sums / (centroid_counts[:, None] + 1)
        
    if path_to_save == None:
        pass
    else:
        torch.save(model.state_dict(), path_to_save)
        
    return model, centroids

In [167]:
opts = {
    'vocab_size': 20000,
    'emb_dim': 512
}

In [168]:
num_gpus = torch.cuda.device_count()
if num_gpus > 0:
    current_device = 'cuda'
else:
    current_device = 'cpu'

model = neuralNetBow(opts['vocab_size'], opts['emb_dim']).to(current_device)

In [169]:
# model = neuralNetBow(opts['vocab_size'], opts['emb_dim'])
centroids = centroid_init(2, opts['emb_dim'],ground_truth_dataloader, model)
criterion = KMeansCriterion(1)
optimizer = torch.optim.Adam(model.parameters(), 0.01, amsgrad=True)

In [170]:
centroids

tensor([[ 0.4438, -0.0525,  0.1054,  ...,  0.1595,  0.1759,  0.5402],
        [ 0.3969, -0.0488,  0.1069,  ...,  0.2194,  0.1766,  0.5496]],
       grad_fn=<CloneBackward>)

In [172]:
current_device

'cpu'

In [173]:
train_model(model, centroids, criterion, optimizer, train_loader, val_loader)








0it [00:00, ?it/s]






1it [00:00,  2.41it/s]

Average training Loss at batch 0 : 36.343









2it [00:00,  2.42it/s]






3it [00:01,  2.41it/s]






4it [00:01,  2.30it/s]






5it [00:02,  2.34it/s]






6it [00:02,  2.32it/s]






7it [00:03,  2.32it/s]






8it [00:03,  2.26it/s]






9it [00:03,  2.22it/s]






10it [00:04,  2.19it/s]






11it [00:04,  2.25it/s]






12it [00:05,  2.32it/s]






13it [00:05,  2.33it/s]






14it [00:06,  2.34it/s]






15it [00:06,  2.33it/s]

KeyboardInterrupt: 

In [22]:
x = torch.tensor([2])
print(type(x))

<class 'torch.Tensor'>


In [83]:
x = torch.Tensor([2])
print(type(x))

<class 'torch.Tensor'>
